In [ ]:
!pip install sdv --quiet
from sdv.single_table import GaussianCopulaSynthesizer

!pip install py-synthpop --quiet

#Load library
import pandas as pd
import numpy as np

from numpy import std
from math import floor
from random import choices
import math

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_breast_cancer

from collections import Counter

import matplotlib.pyplot as plt

import pickle

import time

from imblearn import FunctionSampler  # to use a idendity sampler
from imblearn.under_sampling import TomekLinks,EditedNearestNeighbours,RandomUnderSampler
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, RandomOverSampler, SVMSMOTE,SMOTE,KMeansSMOTE,SMOTENC
from imblearn.combine import SMOTEENN,SMOTETomek

from imblearn.pipeline import Pipeline as imbPipeline

from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder

from synthpop import Synthpop

In [ ]:
data,cat_dict,c_col,n_col,c_col_index,int_col=datatype_detect(df)

In [ ]:
data.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,checking_status,credit_history,purpose,...,employment,personal_status,other_parties,property_magnitude,other_payment_plans,housing,job,own_telephone,foreign_worker,class
0,18,3049,1,1,45,1,0.0,2.0,3.0,3.0,...,2.0,0.0,2.0,1.0,2.0,1.0,3.0,0.0,1.0,1.0
1,24,1927,3,2,33,2,0.0,3.0,1.0,2.0,...,0.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,12,640,4,2,49,1,0.0,3.0,3.0,4.0,...,0.0,1.0,2.0,3.0,1.0,1.0,3.0,0.0,1.0,1.0
3,6,2063,4,3,30,1,0.0,0.0,3.0,6.0,...,2.0,2.0,2.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0
4,12,1526,4,4,66,2,0.0,1.0,1.0,9.0,...,3.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0


####Non Deep Generative Models

In [ ]:
##Simple Simulation

def df_distribution(df,c_col):
  distribution={}
  for i in df.columns.to_list():
    if i in c_col:
      distribution[i]=[df[i].value_counts().to_dict(),df[i].value_counts(normalize=True).to_dict()]
    else:
      distribution[i]={
              "count": len(df[i]),
              "mean": float(np.mean(df[i])),
              "std": float(np.std(df[i])),
              "min": float(np.min(df[i])),
              "p25":float(np.percentile(df[i], 25)),
              "p50":float(np.percentile(df[i], 50)),
              "p75":float(np.percentile(df[i], 75)),
              "max": float(np.max(df[i]))}
  return distribution


def num_generator(count,mean,std,min,p25,p50,p75,max):
  COUNT = count
  MEAN = mean
  STD = std
  MIN = min
  P25 = p25
  P50 = p50
  P75 = p75
  MAX = max
  #Positions of the percentiles
  P25_pos = floor(0.25 * COUNT) - 1
  P50_pos = floor(0.5 * COUNT) - 1
  P75_pos = floor(0.75 * COUNT) - 1
  MAX_pos = COUNT -1
  #Count requirement
  v = [0] * COUNT
  #Min requirement
  v[0] = MIN
  #Max requirement
  v[MAX_pos] = MAX

  #Good, we already satisfied the easiest 3 requirements and these are deterministic,
  #there is only one way to satisfy them
  #This will satisfy the 25th percentile requirement
  for i in range(1, P25_pos):
      #We could also interpolate the value from P25 to P50, even adding a bit of randomness.
      v[i] = np.mean(np.random.uniform(MIN,P25))
  v[P25_pos] = P25
  #pandas does some linear interpolation (https://stackoverflow.com/questions/39581893/pandas-find-percentile-stats-of-a-given-column)
  #when calculating percentiles but we can simulate that by letting the next value be also P25
  if P25_pos + 1 != P50_pos:
      v[P25_pos + 1] = P25
  #We do something similar with the other percentiles
  for i in range(P25_pos + 2, P50_pos):
      v[i] = np.mean(np.random.uniform(P25,P50))
  v[P50_pos] = P50
  if P50_pos + 1 != P75_pos:
      v[P50_pos + 1] = P50
  for i in range(P50_pos + 2, P75_pos):
      v[i] = np.mean(np.random.uniform(P50,P75))
  v[P75_pos] = P75
  if P75_pos + 1 != v[MAX_pos]:
      v[P75_pos + 1] = P75
  for i in range(P75_pos + 1, MAX_pos):
      v[i] = np.mean(np.random.uniform(P75,MAX))
  return pd.Series(v)

def cat_generator(population,proba,count):
  v = choices(population=population,weights=proba,k=count)
  return pd.Series(v)

def generate_datetime(min_time, max_time):
    start = pd.to_datetime(min_time)
    end = pd.to_datetime(max_time)
    random_date = start + (end - start) * np.random.random()
    return random_date

def simplesim(real,c_col,int_col,dir):

  n_samples=real.shape[0]

  var_info=df_distribution(real,c_col)

  new_data={}

  start = time.time()
  for i in real.columns:
    if i in n_col:
      count=var_info[i]['count']
      mean=var_info[i]['mean']
      std=var_info[i]['std']
      min=var_info[i]['min']
      p25=var_info[i]['p25']
      p50=var_info[i]['p50']
      p75=var_info[i]['p75']
      max=var_info[i]['max']
      new_data[i]=num_generator(count,mean,std,min,p25,p50,p75,max)
    else:
      population=list(var_info[i][1].keys())
      proba=var_info[i][1].values()
      new_data[i]=cat_generator(population,proba,n_samples)

  synthetic=pd.DataFrame.from_dict(new_data)

  for col in c_col:
    synthetic[col] = synthetic[col].astype('object')

  for col in int_col:
    synthetic[col] = synthetic[col].astype('int')

  end = time.time()

  duration=round(end-start,2)

  print('Perturbation takes ' +str(end-start) + 'to run')

  synthetic.to_csv(dir+"Simulation.csv",index=False)

  return duration

In [ ]:
##SMOTE based model

def SMOTE_based(real,dir,c_col_index,c_col,int_col):

  n_samples=real.shape[0]

  df_real=real.copy()
  df_fake=real.copy()
  df_real['real']=1
  df_fake['real']=0
  df_rff=pd.concat([df_real,df_fake,df_fake])

  y_f=df_rff['real']
  X_f=df_rff.drop(['real'],axis=1)

  ratio=[1]
  i=1

  balancer_names=[
      "SMOTE",
      "SMOTENC",
      "ADASYN",
      "BorderlineSMOTE",
      "SVMSMOTE",
      "SMOTETomek"]
  balancers = [
              SMOTE(sampling_strategy=i,random_state=1),
              SMOTENC(sampling_strategy=i,random_state=1, categorical_features=c_col_index),
              ADASYN(sampling_strategy=i,random_state=1),
              BorderlineSMOTE(sampling_strategy=i,random_state=1),
              SVMSMOTE(sampling_strategy=i,random_state=1),
              SMOTETomek(smote=SMOTE(sampling_strategy=i,random_state=1)),
               ]

  zipped_balancer = zip(balancer_names,balancers)

  durations={}
  for n,o in zipped_balancer:
    print('-----------'+n+'----------------')
    pipe = imbPipeline([('over', o)])
    start = time.time()
    X_o,y_o=pipe.fit_resample(X_f,y_f)
    y_o=y_o.astype('int')
    df_o=np.c_[X_o, y_o]
    df_new=pd.DataFrame(df_o)
    synthetic=df_new.iloc[-n_samples:,:-1]
    synthetic.columns=real.columns
    print(synthetic.shape)
    end = time.time()
    duration=round(end-start,2)

    for col in c_col:
      synthetic[col] = synthetic[col].astype('object')
    for col in int_col:
      synthetic[col] = synthetic[col].astype('int')

    synthetic.to_csv(dir+n+'.csv',index=False)
    print(n+' takes ' +str(duration) + 'to run')
    durations[n]=duration

  return durations

In [ ]:
##Synthpop

# - all dtypes of df are correct ('int', 'float', 'datetime', 'category', 'bool'; no object)
def synth(data,c_col,int_col,dir):

  real=data.copy()
  n_samples=real.shape[0]

  dtypes={}
  for i in real.columns:
    if i in c_col:
      dtypes[i]="category"
    elif i in int_col:
      dtypes[i]="int"
    else:
      dtypes[i]="float"


  spop = Synthpop()
  start = time.time()
  spop.fit(real,dtypes)

  synthetic = spop.generate(n_samples)
  print(synthetic.shape)
  end = time.time()

  duration=round(end-start,2)

  print('synthpop takes ' +str(end-start) + 'to run')

  synthetic.to_csv(dir+"Synthpop.csv",index=False)

  return duration

In [ ]:
## Copula

def copula(data,dir):

  real=data.copy()
  n_samples=real.shape[0]

  start = time.time()
  metadata = SingleTableMetadata()
  metadata.detect_from_dataframe(data=real)


  synthesizer = GaussianCopulaSynthesizer(metadata)
  synthesizer.fit(real)

  synthetic = synthesizer.sample(num_rows=n_samples)
  end = time.time()

  duration=round(end-start,2)
  print('GaussianCopulas takes ' +str(end-start) + ' to fit')

  synthetic.to_csv(dir+"Copula.csv",index=False)

  return duration

In [ ]:
##/content/drive/MyDrive/Experiment/DATA/boston
folds=5
R_sec={}

for i in range(folds):
  print('\n')
  print(f'Fold {i} is currently processing------------------------')
  baseline=pd.read_csv(data_dir+dir+'/synthetic/seed' + str(i) +'/baseline.csv')

  output_dir=data_dir+dir+'/synthetic/seed' + str(i) +'/'

  sec=simplesim(baseline,c_col,int_col,output_dir)
  R_sec[dir+str(i)+'_ss']=sec
  sec=SMOTE_based(baseline,output_dir,c_col_index,c_col,int_col)
  R_sec[dir+str(i)+'_sm']=sec
  sec=synth(baseline,c_col,int_col,output_dir)
  R_sec[dir+str(i)+'_sy']=sec
  sec=copula(baseline,output_dir)
  R_sec[dir+str(i)+'_co']=sec

print('\n')
print('------------------------------------------------------')

####Deep Generative Models

In [ ]:
!pip install ctgan --quiet
from ctgan import CTGAN,TVAE

!pip install sdv --quiet

from sdv.metadata import SingleTableMetadata
from sdv.single_table import CopulaGANSynthesizer

In [ ]:
##CTGAN

def CTGAN_gen(data,c_col,dir):

  real= data.copy()

  start = time.time()
  ctgan = CTGAN(verbose=True)

  ctgan.fit(real, c_col)
  ctgan.save(dir+'CTGAN.pkl')
  ctgan_load = CTGAN.load(dir+'CTGAN.pkl')
  synthetic = ctgan_load.sample(real.shape[0])

  end = time.time()
  duration=round(end-start,2)
  print('model takes ' +str(end-start) + ' to run')
  synthetic.to_csv(dir+'CTGAN.csv',index=False)

  return duration

def TVAE_gen(data,c_col,dir):

  real=data.copy()

  start = time.time()
  tvae = TVAE()

  tvae.fit(real,c_col)
  tvae.save(dir+'TVAE.pkl')
  tvae_load = TVAE.load(dir+'TVAE.pkl')
  synthetic = tvae_load.sample(real.shape[0])

  end = time.time()
  duration=round(end-start,2)
  print('model takes ' +str(duration) + ' to run')
  synthetic.to_csv(dir+'TVAE.csv',index=False)

  return duration

## Copula

def CopulaGAN_gen(data,dir):

  real=data.copy()
  n_samples=real.shape[0]

  start = time.time()
  metadata = SingleTableMetadata()
  metadata.detect_from_dataframe(data=real)


  synthesizer = CopulaGANSynthesizer(metadata)
  synthesizer.fit(real)

  synthetic = synthesizer.sample(num_rows=n_samples)
  end = time.time()

  duration=round(end-start,2)
  print('GaussianCopulas takes ' +str(end-start) + ' to fit')

  synthetic.to_csv(dir+"Copula.csv",index=False)

  return duration



In [ ]:
folds=5

for i in range(folds):
  print('\n')
  print(f'Fold {i} is currently processing------------------------')
  baseline=pd.read_csv(data_dir+dir+'/synthetic/seed' + str(i) +'/baseline.csv')

  output_dir=data_dir+dir+'/synthetic/seed' + str(i) +'/'
  CTGAN_gen(baseline,c_col,output_dir)
  TVAE_gen(baseline,c_col,output_dir)
  CopulaGAN_gen(baseline,output_dir)
print('\n')
print('------------------------------------------------------')


In [1]:
!git clone https://github.com/Team-TUD/CTAB-GAN.git
import sys
sys.path.append('/content/CTAB-GAN/')

Cloning into 'CTAB-GAN'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 108 (delta 40), reused 75 (delta 18), pack-reused 0
Receiving objects: 100% (108/108), 1.05 MiB | 12.19 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
# Importing the model
from model.ctabgan import CTABGAN

In [ ]:
# Specifying the replication number
num_exp = 1
# Specifying the name of the dataset used
dataset = "credit-g"
# Specifying the path of the dataset used
real_path = "/credit-g/synthetic/seed0/baseline.csv"
# Specifying the root directory for storing generated data
fake_file_root = "synthetic"

In [ ]:
# Initializing the synthesizer object and specifying input parameters
# Notice: If you have continuous variable, you do not need to explicitly assign it. It will be treated like
# that by default
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,
                 categorical_columns = c_col,
                 log_columns = [],
                 integer_columns = n_col,
                 problem_type= {"Classification": 'class'},
                 epochs = 300)


# Fitting the synthesizer to the training dataset and generating synthetic data
for i in range(num_exp):
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)